In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import optuna
import json

import tensorflow as tf
from tensorflow.keras.regularizers import L1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization,Activation
from tensorflow.keras import regularizers
# where you save the data 
path = '../Data/BTC/'


In [67]:

# parse x:
def read_json_file(variable_name): 
    # Open the JSON file and read its content
    with open(path + variable_name + '.json', 'r') as file:
        data = json.load(file)
    df = pd.DataFrame(data[variable_name])
    df['x'] = df['x'].astype(int)
    df['date'] = pd.to_datetime(df['x'], unit='ms')
    df[variable_name] = df['y']
    df = df.drop(['x','y'], axis=1)
    return df


In [81]:
def calculate_weekly_average(df): 
    df.set_index('date', inplace=True)
    # Group by month and calculate the average trade volume
    monthly_avg_df = df.resample('W').mean()
    # Reset the index to have 'date' as a column again
    monthly_avg_df.reset_index(inplace=True)
    # Display the resulting DataFrame
    return monthly_avg_df

In [82]:
def create_df_from_column_names(first_column, names): 
    df = calculate_weekly_average(read_json_file(first_column))
    for i in names: 
        price = calculate_weekly_average(read_json_file(i))
        df = df.merge(price, on='date')
    return df


In [84]:
columns = ['difficulty', 'n-unique-addresses', 'trade-volume', 'n-transactions-total', 'cost-per-transaction', 'hash-rate', 
           'miners-revenue', 'n-transactions-excluding-popular', 'n-transactions-per-block', ]
df = create_df_from_column_names('market-price', columns)
df = df[df['market-price'] != 0]
df.reset_index(inplace=True)
df

# df = read_json_file('../Data/BTC/market-price.json', 'market-price')
# # read_json_file('../Data/difficulty.json', 'difficulty')
# df = df.merge(read_json_file('../Data/difficulty.json', 'difficulty'), on='date')
# df = df.merge(read_json_file('../Data/n-unique-addresses.json', 'n-unique-addresses'), on='date')


,index,date,market-price,difficulty,n-unique-addresses,trade-volume,n-transactions-total,cost-per-transaction,hash-rate,miners-revenue,n-transactions-excluding-popular,n-transactions-per-block
0,78,2010-08-22,0.015000,4.433683e+02,411.5,6.720000e+01,1.045610e+05,0.801056,4.185478e-03,3.412500e+02,371.5,3.005954
1,79,2010-08-29,0.070000,5.277183e+02,388.0,0.000000e+00,1.074885e+05,1.871764,4.971524e-03,6.772780e+02,368.0,2.657624
2,80,2010-09-05,0.067143,6.074422e+02,366.0,0.000000e+00,1.101820e+05,1.415564,5.249310e-03,4.830027e+02,324.0,2.020284
3,81,2010-09-12,0.061429,6.361724e+02,310.0,0.000000e+00,1.120570e+05,1.260756,5.134171e-03,4.110066e+02,293.5,2.013205
4,82,2010-09-19,0.068571,7.064922e+02,436.5,0.000000e+00,1.142205e+05,1.465776,6.108049e-03,5.805000e+02,392.5,2.021661
...,...,...,...,...,...,...,...,...,...,...,...,...
712,790,2024-04-14,68808.899286,8.361608e+13,631803.0,5.553273e+08,9.859861e+08,158.122100,6.251834e+08,7.170508e+07,501919.5,2636.344772
713,791,2024-04-21,65410.795000,8.617873e+13,696578.0,5.097638e+08,9.897556e+08,162.531666,6.339960e+08,8.981683e+07,417862.0,2764.992090
714,792,2024-04-28,64544.908571,8.650241e+13,557883.5,2.421773e+08,9.942237e+08,72.425493,6.395156e+08,4.011381e+07,556849.0,3053.069206
715,793,2024-05-05,63686.565714,8.779348e+13,508495.5,3.475803e+08,9.973177e+08,50.814486,6.207114e+08,2.893412e+07,578940.0,3327.091410
